## Initialize all imports

In [1]:
import os
import conda_installer
import pandas as pd
import tensorflow as tf
import numpy as np
from rdkit import Chem
from deepchem.feat.graph_features import atom_features as get_atom_features
import rdkit
import pickle
import copy
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


import importlib
import keras.backend as K
# import tensorflow_addons as tfa
from tensorflow.keras import regularizers, constraints, callbacks

import sys
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam


2025-04-28 14:39:58.994340: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-28 14:39:59.061448: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
No normalization for NumAmideBonds. Feature removed!
No normalization for NumAtomStereoCenters. Feature removed!
No normalization for NumBridgeheadAtoms. Feature removed!
No normalization for NumHeterocycles. Feature removed!
No normaliz

## Read Input data & Change directory

In [2]:
df = pd.read_csv('Datasets/pdbbind_100.csv')
PDBs = pickle.load(open('Datasets/PDBBind_100.pkl', 'rb'))


In [3]:
df = df[df['complex-name'] != '1.00E+66']


In [4]:
len(df)

98

#### PDB PICKLE FILE GENERATION FOR 100 molecule [NOT NEEDED]

In [5]:

# complex_names_df = df['complex-name'].to_numpy()
# PDBs = {}
# from os import listdir
# from os.path import isfile, join
# mypath = 'pdbbind_complex'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# for f in onlyfiles:
#     if f.split('.')[0] in complex_names_df:
#         PDBs.update({f.split('.')[0] : rdkit.Chem.rdmolfiles.MolFromPDBFile(mypath + '/' + f)})
        
# PDBs = {pdb: value for pdb, value in PDBs.items() if value is not None}
# pkl_file = open('PDBBind_100.pkl', 'wb')
# pickle.dump(PDBs, pkl_file)
# pkl_file.close()

## DATA PREPROCESSING df ~ pdb

In [6]:
pdb_keys = set(PDBs.keys())
df_filtered = df[df['complex-name'].isin(pdb_keys)]
len(df_filtered)

98

In [7]:
df_final = df_filtered[['complex-name','pb-protein-vdwaals', 'pb-ligand-vdwaals', 'pb-complex-vdwaals', 'gb-protein-1-4-eel', 'gb-ligand-1-4-eel', 'gb-complex-1-4-eel',
       'gb-protein-eelect', 'gb-ligand-eelec', 'gb-complex-eelec', 'gb-protein-egb', 'gb-ligand-egb', 'gb-complex-egb', 'gb-protein-esurf', 'gb-ligand-esurf', 'gb-complex-esurf','ddg']]

In [8]:
len(df_final)

98

## Data pre-processing

In [9]:
import models.layers_update_mobley as layers
from models.dcFeaturizer import atom_features as get_atom_features
importlib.reload(layers)

<module 'models.layers_update_mobley' from '/home/lthoma21/BFE-Loss-Function/FINAL-PDBBIND-FILES/models/layers_update_mobley.py'>

In [10]:
info = []
for pdb in list(PDBs.keys()):
    info.append(df_final[df_final['complex-name'] == pdb][['pb-protein-vdwaals', 'pb-ligand-vdwaals', 'pb-complex-vdwaals', 'gb-protein-1-4-eel', 'gb-ligand-1-4-eel', 'gb-complex-1-4-eel',
       'gb-protein-eelect', 'gb-ligand-eelec', 'gb-complex-eelec', 'gb-protein-egb', 'gb-ligand-egb', 'gb-complex-egb', 'gb-protein-esurf', 'gb-ligand-esurf', 'gb-complex-esurf']].to_numpy()[0])

In [11]:
def featurize(molecule, info):
    
    atom_features = []
    for atom in molecule.GetAtoms():
        new_feature = get_atom_features(atom).tolist()
        position = molecule.GetConformer().GetAtomPosition(atom.GetIdx())
        new_feature += [atom.GetMass(), atom.GetAtomicNum(),atom.GetFormalCharge()]
        new_feature += [position.x, position.y, position.z]
        for neighbor in atom.GetNeighbors()[:2]:
            neighbor_idx = neighbor.GetIdx()
            new_feature += [neighbor_idx]
        for i in range(2 - len(atom.GetNeighbors())):
            new_feature += [-1]

        atom_features.append(np.concatenate([new_feature, info], 0))
    return np.array(atom_features)

In [12]:
X = []
y = []
for i, pdb in enumerate(list(PDBs.keys())):
    X.append(featurize(PDBs[pdb], info[i]))
    y.append(df_final[df_final['complex-name'] == pdb]['ddg'].to_numpy()[0])


In [13]:
#  To find max number of atoms in the Dataframe after featurizing
# Required to provide max_atom_number for padding the X_train
# x = []
# y=[]
# for i in range(len(X)):
#     shape = X[i].shape
#     x.append(shape[0])
#     y.append(shape[1])
# max(x)


In [14]:
# Split the data into training and testing sets
# Randomly shuffles the data before splitting, ensuring that the training and testing sets are representative of the overall dataset.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=50)


In [15]:
len(X_train), len(X_test), len(y_train), len(y_test)

(78, 20, 78, 20)

## Helper Functions

In [16]:
import models.layers_update_mobley as layers
import importlib
importlib.reload(layers)

from tensorflow.keras.callbacks import EarlyStopping

class PGGCNModel(tf.keras.Model):
    def __init__(self, num_atom_features = 36, r_out_channel = 20, c_out_channel = 1024):
        super().__init__()
        self.ruleGraphConvLayer = layers.RuleGraphConvLayer(r_out_channel, num_atom_features, 0)
        self.ruleGraphConvLayer.combination_rules = []
        self.conv = layers.ConvLayer(c_out_channel)
        self.dense1 = tf.keras.layers.Dense(32, activation='relu', name='dense1')
        self.dense5 = tf.keras.layers.Dense(16, name='relu')
        self.dense6 = tf.keras.layers.Dense(1, name='dense6')
        self.dense7 = tf.keras.layers.Dense(1, name='dense7',
                 kernel_initializer=tf.keras.initializers.Constant([.3, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, -1]),
                 bias_initializer=tf.keras.initializers.Zeros())
        self.all_layer_1_weights = []
        
    def addRule(self, rule, start_index, end_index = None):
        self.ruleGraphConvLayer.addRule(rule, start_index, end_index)
    
    def set_input_shapes(self, i_s):
        self.i_s = i_s

    def call(self, inputs):
        physics_info = inputs[:,0,38:]
        x_a = []
        for i in range(len(self.i_s)):
            x_a.append(inputs[i][:self.i_s[i], :38])
        x = self.ruleGraphConvLayer(x_a)
        self.all_layer_1_weights.append(self.ruleGraphConvLayer.w_s)
        x = self.conv(x)
        x = self.dense1(x)
        x = self.dense5(x)
        model_var = self.dense6(x)
        merged = tf.concat([model_var, physics_info], axis=1)
        out = self.dense7(merged)
        return out

m = PGGCNModel()
m.addRule("sum", 0, 32)
m.addRule("multiply", 32, 33)
m.addRule("distance", 33, 36)

import keras.backend as K
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred[0] - y_true))) + K.abs(1 / K.mean(.2 + y_pred[1]))
def pure_rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

opt = tf.keras.optimizers.Adam(learning_rate=0.005)
m.compile(loss=pure_rmse, optimizer=opt)

X_train, y_train = X, y


input_shapes = []
for i in range(len(X_train)):
    input_shapes.append(np.array(X_train[i]).shape[0])
m.set_input_shapes(input_shapes)
for i in range(len(X_train)):
    if X_train[i].shape[0] < 12000:
        new_list = np.zeros([12000 - X_train[i].shape[0], 53])
        X_train[i] = np.concatenate([X_train[i], new_list], 0)
X_train = np.array(X_train)
y_train = np.array(y_train)
early_stop = EarlyStopping(
    monitor='loss',           
    patience=10,              
    restore_best_weights=True, 
    min_delta=0.001,          
    verbose=1                 
)
hist = m.fit(X_train, y_train, epochs = 400, batch_size=len(X_train), callbacks=[early_stop])

2025-04-28 14:40:45.245166: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-04-28 14:40:45.245197: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (curie.cluster): /proc/driver/nvidia/version does not exist
2025-04-28 14:40:45.245847: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/400
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
1/1 [==============================] - 182s 182s/step - loss: 47.0002
Epoch 2/400
1/1 [==============================] - 70s 70s/step - loss: 663.0574
Epoch 3/400
1/1 [==============================] - 67s 67s/step - loss: 260.3631
Epoch 4/400
1/1 [==============================] - 66s 66s/step - loss: 270.2631
Epoch 5/400
1/1 [==============================] - 67s 67s/step - loss: 397.5255
Epoch 6/400
1/1 [==============================] - 64s 64s/step - loss: 309.5937
Epoch 7/400
1/1 [==============================] - 66s 66s/step - loss: 93.1966
Epoch 8/400


KeyboardInterrupt: 

In [19]:
X_train[0].shape

(4196, 53)

In [ ]:
# y_pred = m.predict(X_test, batch_size=53)
# print("Predictions:", y_pred)

In [ ]:
import copy
input_shapes = []
for i in range(len(X_test)):
    input_shapes.append(np.array(X_test[i]).shape[0])
m.set_input_shapes(input_shapes)

for i in range(len(X_test)):
    if X_test[i].shape[0] < 12000:
        new_list = np.zeros([12000 - X_test[i].shape[0], 53])
        X_test[i] = np.concatenate([X_test[i], new_list], 0)
X_test = np.array(X_test)
x_c = copy.deepcopy(X_test)
y_test = np.array(y_test)
y_pred_test = m.predict(X_test) 
y_pred_test = np.array(y_pred_test)

y_difference = np.mean(np.abs(np.abs(y_test) - np.abs(y_pred_test)))
eval = m.evaluate(X_test, y_test)
print("The mean absolute difference between y_tru & y_pred is : {}" .format(str(y_difference)))
print(y_pred_test)
print(y_test)
